# **GRUPO N° 2**

### Materia LABORATORIO 2

### Profesores
#### 1) Andres Javier ARANEO
#### 2) Tomas LANZA

### Alumnos
#### 1) Leandro BOISSELIER
#### 2) Juan CZYRKA
#### 3) Pablo CABRAL
#### 4) Franco SONZOGNI

### Repositorio
#### [Repositorio Grupo 2] https://github.com/lmbsoft/mcd24-labo2/tree/main

# **CASO**

Esta solución busca predecir la rapidez con la que se adoptará una mascota, basándose en datos tabulares, descripciones y fotos. El objetivo es entregar herramientas que ayuden a refugios y rescatistas a mejorar los perfiles de sus mascotas, aumentando las posibilidades de adopción y reduciendo el sufrimiento animal.

## 1. Importar las Librerías Necesarias

In [ ]:
# Las librerías usadas en los distintos procesos que ejecutamos a lo largo del desarrollo fueron:

import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import WordCloud
import TextBlob
import sklearn
import json
import nltk
import lightgbm as lgb
import optuna
import joblib
# Librería provista por los profesores para poder generar una matriz de confusión custom.
from UA_MDM_LDI_II.tutoriales.utils import plot_confusion_matrix
import torch
import sys
import shutil
import time
import copy
import datetime
import tqdm
import torchvision
import utils
from augment.autoaugment import ImageNetPolicy
from augment.cutout import Cutout
import gc
from plotly import express as px

## 2. Leer los Datos


## Los datos de partida para iniciar el proceso, son los provistos mediane la plataforma Kaggle.

### Descripciones de archivos
1) **train.csv**: datos tabulares/de texto para el conjunto de entrenamiento
2) **breed_labels.csv**: contiene el tipo y el nombre de la raza para cada BreedID. El tipo 1 es perro y el 2 es gato.
3) **color_labels.csv**: contiene el nombre del color para cada ID del color
4) **state_labels.csv**: contiene el nombre del estado para cada ID de estado

### Adicional a estos datos tabulares se nos provee de la siguiente información estructurada y semiestructurada (json):

1) Imágenes
En el caso de las mascotas que tengan fotos, tenemos un set de archivos provistos por Kaggle en el formato "PetID-ImageNumber.jpg". Siempre la imagen 1 es la foto de perfil (predeterminada) establecida para la mascota. Por motivos de privacidad, se han ocultado los rostros, los números de teléfono y los correos electrónicos.

2) Metadatos de la imagen

    Hemos procesado las imágenes a través de la API Vision de Google, lo que nos proporciona un análisis de anotaciones faciales, anotaciones de etiquetas, anotaciones de texto y propiedades de la imagen.

    Algunas propiedades no existirán en el archivo JSON si no están presentes, por ejemplo, la anotación de rostros. La anotación de texto se ha simplificado a solo una entrada de la descripción de texto completa (en lugar del resultado JSON detallado desglosado por caracteres y palabras individuales). Los números de teléfono y los correos electrónicos ya están anonimizados en la anotación de texto.

    Referencia de la API de Google Vision:
    https://cloud.google.com/vision/docs/reference/rest/v1/images/annotate

3) Datos de sentimientos
    Hemos ejecutado la descripción de cada perfil de mascota a través de la API de lenguaje natural de Google, lo que proporciona un análisis de sentimientos y entidades clave. Utilizamos esta información complementaria para el análisis de la descripción de la mascota. Algunas descripciones la API no las pudo analizar. Por lo tanto, encontramos menos archivos de sentimientos que filas en el conjunto de datos.

    El formato del nombre del archivo es PetID.json .

    Referencia de la API de lenguaje natural de Google:
    https://cloud.google.com/natural-language/docs/basics


### En la medida que el proceso avanza, fuimos creando nuevos lotes de datos. Estos son:

1) **train_data_ext.csv** --> Agrega nuevos atributos calculados.
2) **train_data_fe.csv** --> Agrega nuevos atrivutos mediante diferentes métodos de Feature Engineering.
3) **train_data_fe_cleaned.csv** --> Realiza ajustes sobre los nuevos features.
4) **train_data_fe_text_llm.csv** --> Agrega features obtenidos mediante técnicas de LLM.


## 3. Proceso

### Nuestro proceso de trabajo se puede seguir desde los siguientes notebooks:

####    1) Flujo inicia en el EDA: **grupo2-EDA-ii.ipynb**
https://github.com/lmbsoft/mcd24-labo2/blob/main/grupo2-EDA-ii.ipynb
####    1b) Proceso de extracción de características mediante LLM:  **01_FE_llms_ollama.ipynb** (Se corre por única vez, dado que el tiempo de procesamiento lleva +de 8 hs y no es necesario reprocesar cada vez que re entrenamos el modelo)
https://github.com/lmbsoft/mcd24-labo2/blob/main/01_FE_llms_ollama.ipynb
####    2) Proceso de FE: **grupo2-FE.ipynb**
https://github.com/lmbsoft/mcd24-labo2/blob/main/grupo2-FE.ipynb
####    3) Entrenamiento de tabulares: **grupo2-Train-Tabulares.ipynb**
https://github.com/lmbsoft/mcd24-labo2/blob/main/grupo2-Train-Tabulares.ipynb
####    4) Entrenamiento resnet: **grupo2-Resnet50_3_train_augment.ipynb**
https://github.com/lmbsoft/mcd24-labo2/blob/main/grupo2-Resnet50_3_train_augment.ipynb
####    5) Integración / Ensamble de modelos: **grupo2-Resnet50_2_integrar_fe.ipynb**
https://github.com/lmbsoft/mcd24-labo2/blob/main/grupo2-Resnet50_2_integrar_fe.ipynb


### CONSIDERACIONES FINALES
Agregar el texto en el caso de que aplique.

## 4. EDA y Pre Procesamiento de datos

### Pasos Ejecutados:
#### 1) Carga de Datos:
        Cargamos los archivos train.csv, StateLabels.csv, ColorLabels.csv y BreedLabels.csv.
        Mostramos las primeras filas de cada archivo para verificar su contenido.

#### 2) Revisión de Datos Faltantes:
        Calculamos los valores faltantes y su porcentaje en el dataset de entrenamiento.

#### 3) Análisis exploratorio general de Variables Categóricas:
        Realizamos gráficos de conteo para variables categóricas como Type, Gender, Color1, Color2, Color3, MaturitySize, FurLength, Vaccinated, Dewormed, Sterilized, Health, State.

#### 4) Análisis exploratorio general de Variables Numéricas:
        Realizamos histogramas para variables numéricas como Age, Quantity, Fee, VideoAmt, PhotoAmt.

#### 5) Análisis de Correlación:
        Calculamos y visualizamos las correlaciones entre las variables numéricas y AdoptionSpeed.
        Identificamos las palabras más positivamente y negativamente correlacionadas con AdoptionSpeed utilizando funciones de la librería TF-IDF.

#### 6) Creación de Nuevas Características:
        Creamos nuevas características basadas en la presencia de palabras importantes en la descripción de las mascotas.

#### 7) Visualización de la Presencia de Palabras:
        Realizamos boxplots para visualizar la presencia de palabras importantes por tipo de mascota y AdoptionSpeed.

### CONSIDERACIONES FINALES

#### Sobre Datos faltantes
Encontramos datos faltantes en la columna nombre, esto por lo que vimos está asociado a los casos en donde el registro describe a una población de mascotas, a casos donde se desconoce el nombre o donde la mascota es un cachorro y probablemente se busca que el potencial nuevo propietario pueda definir su nombre. **En el grupo decidimos no hacer ningún tipo de transformación sobre estos faltantes.**

#### Outliers
Para abordar la problemática de valores atípicos consideramos diferentes técnicas, y terminamos seleccionando la opción que nos ofrece la robustez implícita que tiene el modelo de LGBM.

#### Lematización
Aplicamos NLTK + TFIDF para analizar la correlación de lemas con la clase target. El resultado del análisis lo aplicamos en el proceso de FE.





## 5. Feature Engeneering mediante la aplicación de LLM
    https://github.com/lmbsoft/mcd24-labo2/blob/main/01_FE_llms_ollama.ipynb


### Pasos Ejecutados:

#### 1) Unión de Datos con Etiquetas:
        Unimos los datos de train.csv con las etiquetas de razas, colores y estados.

#### 2) Mapeo de Valores Numéricos a Descripciones:
        Creamos diccionarios para mapear valores numéricos a descripciones textuales para las variables Gender, MaturitySize, FurLength, Vaccinated, Dewormed, Sterilized y Health.

#### 3) Análisis de Coherencia:
        Definimos una función para generar preguntas y obtener respuestas de coherencia utilizando ollama.

#### 4) Aplicación del Análisis de Coherencia:
        Definimos una función para aplicar el análisis de coherencia a todo el dataset.

#### 5) Aplicación del Análisis de Coherencia al Dataset:
        Aplicamos el análisis de coherencia al dataset extendido.

### Se guardó el nuevo dataset con las características de coherencia en un archivo CSV.

### CONSIDERACIONES FINALES

En este primer proceso de FE aplicamos modelos de LLM para crear nuevas características relacionadas con la coherencia de la descripción y los datos tabulares. También generamos características que buscan obtener clasificaciones tabuladas para lograr una mejor descripción del individuo.

## 5b. Feature Engeneering
    https://github.com/lmbsoft/mcd24-labo2/blob/main/grupo2-FE.ipynb
    


### Pasos Ejecutados:

#### 1) Creación de nuevas características:
        Creamos nuevas características producto del análisis de EDA. Estas se fundan en diferentes aspectos, como: Correlaciones con la velocidad de adopción. Correlaciones entre atributos que agrupan características similares (SALUD), agrupación de atributos que describen aspectos de las mascotas como color y raza.

#### 2) Limpieza y Normalización de Texto:
        Aplicamos funciones para limpiar y normalizar los textos de las descripciones de las mascotas. Posteriormente trabajamos sobre el análisis de sentimiento que encontramos en los archivos Json procesados mediante la API de Google.
        También se aplican técnicas de lematización para realizar cruces entre la frecuencia de las palabras y la velocidad de adopción y creamos nuevos atributos.

#### 3) Análisis de Coherencia:
        Implementamos un análisis de coherencia utilizando un modelo LLM (phi3 de ollama) para evaluar descripciones y características. Ver ítem anterior.

Enfocamos la exploración de datos y la ingeniería de características en enriquecer el dataset con información descriptiva de razas, colores y características geográficas y demográficas de los estados en los cuales se encuentran los refugios. Creamos nuevas características como por ejemplo "IsPureBreed", "Overall_Health", etc. 
Finalmente realizamos un merge con las características del análisis de coherencia de las descripciones de las mascotas, lo cual, a nuestro entender mejora significativamente la calidad y utilidad del dataset. ​

### CONSIDERACIONES FINALES
Sumamos las siguientes características manuales y luego del análisis EDA:

   - `IsPureBreed`: Indica si la mascota es de raza pura.
   - `AgeInYears`: Edad de la mascota en años.
   - `Healthy`: Indica si la mascota está en buenas condiciones de salud.
   - `Overall_Health` : Un indicador categórico de las condiciones de salud
   - `IsFree`: Indica si la adopción es gratuita.
   - `Breed1_Rarity`: Rareza de la raza primaria.
   - `ColorQuantity`: Cantidad de colores
   - `TotalMedia`: Cantidad de imágenes y videos
   - `StatePopulation`: Población aproximada del estado.
   - `IsPeninsular`: Indica si el estado es peninsular.
   - `QuantityCategory`: Categoría de la cantidad de animales en la publicación.
   - `QuantityLog`: Transformación logarítmica de la cantidad de animales.

Sumamos las siguientes características desde el proceso de análisis de sentimiento que describimos en la notebook **grupo2-FE**:

   - 'SentimentMagnitude'
   - 'SentimentScore'
   - 'SentimentLanguage'
   - 'SentimentInterpretation'

En el grupo decidimos dejar todas estas características luego de realizar un análisis de feature importance, en el cual vimos que tienen impacto en la construcción del modelo final.

![Feature Importance Team 2](https://drive.google.com/uc?id=1yZ2XAGDeVa6G4JYhMoME_FOrzUxtgqgQ)


## 6. Modelado predictivo usando **"LightGBM"**


### El proceso de modelado predictivo incluyó la carga y preparación de datos, optimización de hiperparámetros con Optuna, entrenamiento y evaluación de modelos LightGBM, y tratamos de identificar los mejores resultados, buscamos darle un enfoque riguroso y lo mas detallado posible. Intentamos obtener modelos de alta precisión y minimizar el overfitting, asumiendo la complejidad que esto representa.

### El proceso aplicado fue el siguiente:

#### 1) División de Datos en Entrenamiento y Prueba:
        Dividimos los datos en conjuntos de entrenamiento y prueba utilizando train_test_split.

#### 2) Definición de Funciones de Evaluación y Objetivo:
        Definimos funciones de evaluación como **cohen_kappa_score, accuracy_score y balanced_accuracy_score**.
        Cionfiguramos la función objetivo para Optuna, que incluye el entrenamiento del modelo y la evaluación del rendimiento utilizando Kappa cuadrática.

#### 3) Configuración y Ejecución de Optuna para la Optimización de Hiperparámetros:
        Creamos un estudio de Optuna para la optimización de hiperparámetros del modelo LightGBM.
        Definimos el almacenamiento de artefactos de Optuna y ejecutamos la búsqueda de los mejores hiperparámetros.

#### 4) Entrenamiento del Modelo:
        Entrenamos el modelo con los mejores hiperparámetros utilizando un esquema de **validación cruzada StratifiedKFold**.

#### 5) Evaluación del Modelo:
        Evaluamos el modelo en los conjuntos de entrenamiento y prueba, calculando métricas como Kappa cuadrática, precisión, precisión balanceada y reporte de clasificación.
        Agregamos las matrices de confusión para analizar y entender el rendimiento.

#### 6) Almacenamiento de Resultados:
        Alamacenamos los resultados de cada fold en un DataFrame, incluyendo las métricas de evaluación y aspectos relacionados al overfitting.
        Ordenamos y mostramos los resultados para identificar los mejores modelos.

#### 7) Análisis de los Mejores Trials de Optuna:
        Extraemos y analizamos los 30 mejores trials de Optuna, entrenando y evaluando modelos con los mejores hiperparámetros.
        Imprimimos los resultados de los mejores trials para comparación.

### CONSIDERACIONES FINALES
La primer ejecución del entrenamiento utilizando los valores baseline de hiperparámetros nos arrojó un índice de Kappa: **0.335**
A continuación ejecutamos varios estudios con optuna, analizamos los rendiminetos y finalmente nos quedamos con el estudio llamado **"05 - LGB Multiclass CV 06"**
El modelo final, con optimización de hiperparámetros mediante optuna nos arrojó en train un Kappa de: **0.384**
Este mismos modelo en test nos da un Kappa de: **0.331**
La matriz de confusión nos indica que el modelo en general tiene mayor precisión para predecir la **clase 4**, una presición aceptable para las clases **1,2 y 3** y no se logran mejoras para la **clase 0**.
En resumen, pensando en el modelo de negocio, consideramos que es válido que el modelo tenga una mejor presición para los individuos que permanecen mas tiempo en el refugio.
En el análisis de feature importance observamos que las características agregadas en el proceso de FE tuvieron un muy buen Score. Por ejemplo:

- Breed1_Rarity
- TotalMedia
- StatePopulation
- Is_Calm
- Contains_Adoption

La característica de mayor score es:

- SentimentMagnitude


## 7. Modelado en base a imágenes (Datos no estructurados)

### En este proceso de entrenamiento en el cual procesamos las imágenes asociadas a las mascotas, aplicamos redes neuronales y esto involucró la carga y configuración de un modelo **ResNet50 preentrenado**, usamos técnicas de aumento de datos, optimizamos hiperparámetros utilizando Optuna, y gestionamos recursos de GPU, asegurando así un entrenamiento optimizado para la clasificación de imágenes.

### El proceso aplicado es el siguiente:

#### 1) Verificación de Disponibilidad de GPU:
        Comprobamos de la disponibilidad de GPU para el entrenamiento.

#### 2) Importación de Librerías:
        Importamos las librerías esenciales para el manejo de datos, modelos, optimización y visualización, incluyendo torch, optuna, sklearn y joblib.

#### 3) Instalación de Paquetes Necesarios:
        Instalación del paquete kaleido para visualización.

#### 4) Carga y Configuración del Modelo:
        Importamos el modelo preentrenado ResNet50.
        Modificamos la última capa para adaptarse a la clasificación de 5 clases.
        Configuramos el dispositivo para usar CUDA si está disponible.
        Definimos el criterio de pérdida como CrossEntropyLoss.
        Configuramos el optimizador SGD con parámetros de learning rate y momentum.

#### 5) Definición de Transformaciones y Data Augmentation:
        Definición de transformaciones para aumentar el dataset, incluyendo políticas de AutoAugment y Cutout.

#### 6) Carga y División de Datos:
        Cargamos los datos de entrenamiento y validación, y aplicación de las transformaciones definidas.
        Dividimos los datos en conjuntos de entrenamiento y validación.

#### 7) Definición de la Función de Entrenamiento:
        Implementamos una función para entrenar y validar el modelo, incluyendo la configuración del optimizador y el ciclo de entrenamiento.

#### 8) Entrenamiento del Modelo:
        Ejecutamos el entrenamiento del modelo con los datos cargados y transformados.
        Guardamos del mejor modelo encontrado durante el entrenamiento.

#### 9) Optimización de Hiperparámetros con Optuna:
        Definimos la función de entrenamiento para Optuna, sugiriendo valores para epochs, learning rate y momentum.
        Creamos y ejecutamos el estudio de Optuna para optimizar los hiperparámetros del modelo.

### CONSIDERACIONES FINALES
- Hicimnos un primer exprimento sin perturbar las imágenes y obtuvimos un Kappa de: **0.293**
- Posterioremente aplicamos una perturbación de las imágenes durante el entrenamiento y logramos un Kappa de: **0.333**



## 8. Desarrollar modelo ensamblado (Opcional)

### Mediante este proceso de ensamblaje de modelos, buscamos combinar de forma efectiva las predicciones de **LightGBM (basado en datos tabulares)** y **ResNet (basado en imágenes)** para mejorar la precisión de la predicción. Pretendemos aprovechar las fortalezas de ambos enfoques, y utilizamos matrices de confusión y el coeficiente de Kappa cuadrática para analizar y verificar si obtenemos mejoras en la precisión del modelo combinado.

### El proceso aplicado es el siguiente:

#### 1) Carga de Estudios de Optuna:
        Tomamos los estudios de Optuna para los modelos LightGBM y ResNet previamente optimizados y guardados.

#### 2) Carga de Resultados de Modelos:
        Cargamos los resultados de predicción de los modelos LightGBM y ResNet desde los artefactos de Optuna.

#### 3) Fusión de Resultados:
        Fusionamos los resultados de predicción de ambos modelos en un solo DataFrame.
        Renombramos columnas para claridad (lgb_pred_score y resnet_pred_score).

#### 4) Manejo de Valores Nulos:
        Reemplazamos valores nulos en las predicciones del modelo ResNet con vectores de ceros.

#### 5) Cálculo de Predicciones Combinadas:
        Calculamos las puntuaciones combinadas (blend_pred_score) sumando las predicciones de LightGBM y ResNet.

#### 6) Determinación de Clases Predichas:
        Determinamos las clases predichas para cada modelo (lgb_pred y resnet_pred) y para la combinación de ambos (blended_pred).

#### 7) Visualización de Matrices de Confusión:
        Visualizamos las matrices de confusión para evaluar el rendimiento de los modelos individuales y del modelo combinado.
        Cálculamos y visualizamos el coeficiente de Kappa cuadrática para cada conjunto de predicciones.

### CONSIDERACIONES FINALES
Para las puntuaciones combinadas usamos la estrategia de suma de las puntuaciones de predicción de ambos modelos para cada clase y selecciona la clase con la puntuación más alta como la predicción final.
El valor de Kappa obtenido para el modelo ensamblado es de: **0.336**

## 9. Valoración del modelo/conclusiones

1) Observamos que el ensamble del modelo tabular con el de imágenes nos permitió obtener una mejora en le índice de Kappa en comparación a los modelos individuales.
2) Pudimos comprobar durante la experimentación, tanto en el modelo tabular como el de imágenes que aplicando estrategias de FE como la selección de umbrales de busqueda para optimizar HP va aportando mejoras en el rendimiento de los modelos.
3) También pudimos comprobar que al incluir perturbaciones en las imágenes procesadas por el modelo resnet logramos mejores rendimientos.
4) Podemos concluir que si bien el modelo final ensamblado no logró un valor de Kappa alto (menor a 0.6), el mismo es aceptable para este caso de negocio.
5) Confiamos que todas las técnicas aprendidas nos aportarán mayores herramientas para afrontar los desafíos del futuro en el campos de la ciencia de datos.
